# Preliminaries

In [31]:
import os
#os.environ["PATH"]="/opt/conda/envs/python2/bin:"+os.environ["PATH"]

Check the version of Python being used. 

In [32]:
!python --version

Python 2.7.12


Specify the parent directory where you want to store the Cactus directory. Once Cactus is built, its contents cannot be easily moved without breaking things. If the Cactus directory is later moved, one has to repeat the following steps to rebuild it from its new location.

In [5]:
etk_dir = '/home/ejwest/ETK'

Change the working directory to the parent directory specified above.

In [6]:
%cd $etk_dir

/home/ejwest/ETK


![cactus](http://cactuscode.org/global/images/cactuslogo.png)

# Compiling Cactus!

Step 1 is to download the code. Cactus uses a script named "GetComponents" to find and prepare all the source code modules that it needs for a given installation. The GetComponents script can be downloaded with a simple invocation of curl. To download, uncomment and execute the following lines.

In [7]:
#%cd $etk_dir
#!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_02/GetComponents

Step 2 is to download your thornlist. In this tutorial, we are going to use the Einstein Toolkit, a collection of thorns designed to solve the Einstein equations numerically. To download the Einstein Toolkit thornlist, uncomment and execute the following lines.

In [8]:
#%cd $etk_dir
#!curl -kLO https://bitbucket.org/einsteintoolkit/manifest/raw/ET_2018_02/einsteintoolkit.th

You can view a file in the notebook by using the "magic" command "%pycat filename". However, %pycat tries to highlight syntax as if the file is written in python. In those cases you can simply use "%cat filename." Unfortunately, unlike %pycat, %cat leaves the contents of the file on the screen.

To view the downloaded thornlist using %pycat, uncomment and execute the following lines. Note that at the top of the file is "DEFINE_ROOT = Cactus". This means that Cactus, and all its thorns, will be checked out under that directory.

In [39]:
%cd $etk_dir
%pycat einsteintoolkit.th

/home/ejwest/ETK


Step 3 is to checkout the components listed in the thornlist. We do this with the GetComponents command.
Before we can execute it, however, we need to turn on its execute bit. To run GetComponents, uncomment and execute the following lines.

In [11]:
#%cd $etk_dir
#!chmod a+x GetComponents
#!echo no|./GetComponents --update einsteintoolkit.th

In what follows, it will be useful to have a shortcut variable for the Cactus directory. All simulations are launched from this directory. To create a shrtcut variable, edit and execute the following line.

In [12]:
cactus_dir = '/home/ejwest/ETK/Cactus'

Move to the Cactus directory.

In [13]:
%cd $cactus_dir

/home/ejwest/ETK/Cactus


<img src="http://simfactory.org/global/images/factory.png" width="200px">

# Simfactory
Cactus is normally built with a tool called Simfactory. Simfactory, in turn, will call make.
Before it can work, however, it needs to be configured. To configure Simfactory, edit and execute the following lines. Replace the email address in the command below with yours. The email address isn't sent anywhere, all it's used for is allowing Cactus to send job change state notifications to you.

In [14]:
#%cd $cactus_dir
#!./simfactory/bin/sim setup-silent --setup-email=ewest@d.umn.edu 

This has created a user configuration file. This file contains contains defaults that are used any time you run simfactory. In addition, a machine-specific configuration file has been created in teh machine database (mdb). This file is located in "simfactory/mdb/machines". Your machine name can be identified using the following.

In [15]:
!hostname

newton


Next create a shortcut variable for the mdb file. Change the name of the .ini file in the path below to your machine name, returned by the hostname command above.

In [17]:
mdb_file = './simfactory/mdb/machines/newton.ini'
!echo $mdb_file

./simfactory/mdb/machines/newton.ini


One of the things you may want to change in your mdb file is the directory where simulation output data are dumped. By default, a folder named "simulations" is created in your home directory for this purpose. If you want output data to be printed elsewhere, change the "basedir" entry in the mdb file. You may also want to change "ppn", "max-num-threads", and "num-threads" to the number of physical cores on your machine. If you machine supports hyperthreading, add a line "num-smt = 2".

To view the mdb file using %pycat, uncomment and execute the following lines.

In [18]:
#%cd $cactus_dir
#%pycat $mdb_file

To load the mdb file for your machine, uncomment and execute the following line.

In [19]:
#%load $mdb_file

Cut and paste the file contents above into the following cell, adding the magic command "%%writefile $mdb_file" at the top. This allows you to edit the mdb file directly from the notebook interface. When your edits are finished, execute the cell. The contents will overwrite your machine's mdb file.

In [20]:
#%%writefile $mdb_file

At long last, we are ready to actually build Cactus. Cactus can often figure out what compilers and build
options to use automatically, but in some cases it is necessary to specify it by hand (you can do this by adding --optionlist=centos.cfg to the build command below). The file containing
this information is called the Option List. You might want to take a look at it.

To start from a clean slate and remove any previous configurations, uncomment and execute the following lines.

In [21]:
#%cd $cactus_dir
#!rm -rf configs

This is the command to build Cactus using our thornlist. As written, it will build in parallel using two processes. That's what the -j option does. If all goes well, this step takes a while. Go get a coffee and come back in 30 minutes or so! To proceed to build Cactus, uncomment and execute the following lines.

In [22]:
#%cd $cactus_dir
#!time ./simfactory/bin/sim build -j2 --thornlist=thornlists/einsteintoolkit.th 

The build command creates a configuration called "sim". It is found in the "configs/sim" directory. Let's move to this directory and list its contents.

In [40]:
%cd $cactus_dir/configs/sim
%ls

/home/ejwest/ETK/Cactus/configs/sim
BUILD-ID    SubmitScript  config-data/    lib/
CONFIG-ID   ThornList     config-info     piraha/
OptionList  bindings/     configjar.git/  properties.ini
RunScript   build/        datestamp.o     scratch/


Notice that one of the files in this directory is the ThornList. It contains the list of thorns Cactus will compile. If you wish to add or remove a thorn from your configuration, you can do it by editing this file. 
However, by doing so you risk confusing yourself by forgetting what you've done. Proceed at your own risk! To view the current ThornList using %pycat, uncomment and execute the following line.

In [41]:
%pycat ThornList

The "OptionList" file contains all the configuration options (the things you saw in centos.cfg). Unlike the ThornList file, however, changing this file will have no effect. If you wish to change your configuration options without starting over from scratch, you should edit the file "configs/sim/config-data/make.config.defn." To view this file using %pycat, uncomment and execute the following line.

In [42]:
%pycat config-data/make.config.defn

# Running Cactus!

Finally, let's run Cactus. First move to the Cactus directory.

In [33]:
%cd $cactus_dir

/home/ejwest/ETK/Cactus


Below we are going to run a simulation that solves the TOV equations for a static neutron star. We will use MPI and run on two processes. You can edit the parameter file below and hit shift-Enter to write it to disk. The special sequence "%%writefile filename" at the top makes this possible. Alternatively, you can load an existing file by putting the special sequence "%load filename" at the top of a cell and hitting shift-Enter.

In [17]:
%%writefile par/static_tov_small.par
## mclachlan tov_static
ActiveThorns = "Time MoL"
ActiveThorns = "Coordbase CartGrid3d Boundary StaticConformal"
ActiveThorns = "SymBase ADMBase TmunuBase HydroBase InitBase ADMCoupling ADMMacros"
ActiveThorns = "IOUtil Formaline"
ActiveThorns = "SpaceMask CoordGauge Constants LocalReduce aeilocalinterp LoopControl"
ActiveThorns = "Carpet CarpetLib CarpetReduce CarpetRegrid2 CarpetInterp"
ActiveThorns = "CarpetIOASCII CarpetIOScalar CarpetIOHDF5 CarpetIOBasic"

ActiveThorns = "ML_ADMConstraints NaNChecker"

# grid parameters
CartGrid3D::type         = "coordbase"
CartGrid3D::domain       = "full"
CartGrid3D::avoid_origin = "no"
CoordBase::xmin =   0.0
CoordBase::ymin =   0.0
CoordBase::zmin =   0.0
CoordBase::xmax = 240.0
CoordBase::ymax = 240.0
CoordBase::zmax = 240.0
CoordBase::dx   =   12
CoordBase::dy   =   12
CoordBase::dz   =   12

CoordBase::boundary_size_x_lower        = 3
CoordBase::boundary_size_y_lower        = 3
CoordBase::boundary_size_z_lower        = 3
CoordBase::boundary_size_x_upper        = 3
CoordBase::boundary_size_y_upper        = 3
CoordBase::boundary_size_z_upper        = 3
CoordBase::boundary_shiftout_x_lower    = 1
CoordBase::boundary_shiftout_y_lower    = 1
CoordBase::boundary_shiftout_z_lower    = 1
CoordBase::boundary_shiftout_x_upper    = 0
CoordBase::boundary_shiftout_y_upper    = 0
CoordBase::boundary_shiftout_z_upper    = 0




ActiveThorns = "ReflectionSymmetry"

ReflectionSymmetry::reflection_x = "yes"
ReflectionSymmetry::reflection_y = "yes"
ReflectionSymmetry::reflection_z = "yes"
ReflectionSymmetry::avoid_origin_x = "no"
ReflectionSymmetry::avoid_origin_y = "no"
ReflectionSymmetry::avoid_origin_z = "no"



TmunuBase::stress_energy_storage = yes
TmunuBase::stress_energy_at_RHS  = yes
TmunuBase::timelevels            =  1
TmunuBase::prolongation_type     = none

HydroBase::timelevels            = 3

ADMMacros::spatial_order = 4

ADMBase::metric_type     = "physical"

ML_ADMConstraints::timelevels = 3

SpaceMask::use_mask      = "yes"

Cactus::terminate           = "time"
Cactus::cctk_final_time     = 1000

Carpet::domain_from_coordbase = "yes"
Carpet::enable_all_storage       = no
Carpet::use_buffer_zones         = "yes"

Carpet::poison_new_timelevels    = "yes"
Carpet::check_for_poison         = "no"
Carpet::poison_value             = 113

Carpet::init_3_timelevels        = no
Carpet::init_fill_timelevels     = "yes"

CarpetLib::poison_new_memory = "yes"
CarpetLib::poison_value      = 114

# system specific Carpet paramters
Carpet::max_refinement_levels    = 10
driver::ghost_size               = 3
Carpet::prolongation_order_space = 3
Carpet::prolongation_order_time  = 2

CarpetRegrid2::regrid_every = 0
CarpetRegrid2::num_centres  = 1

CarpetRegrid2::num_levels_1 = 5
CarpetRegrid2::radius_1[1]  =120.0
CarpetRegrid2::radius_1[2]  = 60.0
CarpetRegrid2::radius_1[3]  = 30.0
CarpetRegrid2::radius_1[4]  = 15.0


time::dtfac = 0.25
MoL::ODE_Method             = "rk4"
MoL::MoL_Intermediate_Steps = 4
MoL::MoL_Num_Scratch_Levels = 1

# check all physical variables for NaNs
NaNChecker::check_every = 1
NaNChecker::action_if_found = "just warn" #"terminate", "just warn", "abort"
NaNChecker::check_vars = "ADMBase::metric ADMBase::lapse ADMBase::shift HydroBase::rho HydroBase::eps HydroBase::press HydroBase::vel"

## Lapse Condition:  \partial_t alpha = - alpha K
## Shift Condition:  \partial_t beta^i = 0

# Hydro paramters
ActiveThorns = "EOS_Omni"
ActiveThorns = "GRHydro"

HydroBase::evolution_method      = "GRHydro"

GRHydro::riemann_solver            = "Marquina"
GRHydro::GRHydro_eos_type           = "Polytype"
GRHydro::GRHydro_eos_table          = "2D_Polytrope"
GRHydro::recon_method              = "ppm"
GRHydro::GRHydro_stencil            = 3
GRHydro::bound                     = "none"
GRHydro::rho_abs_min               = 1.e-10
#GRHydro::GRHydro = 18 # Tmunu(10), rho,press,eps,w_lorentz,vel, tau
#GRHydro::GRHydro    = 10    # gij(6), alpha, beta(3)


ActiveThorns = "GenericFD NewRad"
ActiveThorns = "ML_BSSN ML_BSSN_Helper"
ADMBase::evolution_method        = "ML_BSSN"
ADMBase::lapse_evolution_method  = "ML_BSSN"
ADMBase::shift_evolution_method  = "ML_BSSN"
ADMBase::dtlapse_evolution_method= "ML_BSSN"
ADMBase::dtshift_evolution_method= "ML_BSSN"

ML_BSSN::timelevels = 3

ML_BSSN::harmonicN           = 1      # 1+log
ML_BSSN::harmonicF           = 1.0    # 1+log
ML_BSSN::evolveA             = 1
ML_BSSN::evolveB             = 0
ML_BSSN::ShiftGammaCoeff     = 0.0
ML_BSSN::AlphaDriver         = 0.0
ML_BSSN::BetaDriver          = 0.0
ML_BSSN::advectLapse         = 0
ML_BSSN::advectShift         = 0

ML_BSSN::MinimumLapse        = 1.0e-8

ML_BSSN::initial_boundary_condition = "extrapolate-gammas"
ML_BSSN::rhs_boundary_condition     = "NewRad"

ML_BSSN::ML_log_confac_bound = "none"
ML_BSSN::ML_metric_bound     = "none"
ML_BSSN::ML_Gamma_bound      = "none"
ML_BSSN::ML_trace_curv_bound = "none"
ML_BSSN::ML_curv_bound       = "none"
ML_BSSN::ML_lapse_bound      = "none"
ML_BSSN::ML_dtlapse_bound    = "none"
ML_BSSN::ML_shift_bound      = "none"
ML_BSSN::ML_dtshift_bound    = "none"



# init parameters
InitBase::initial_data_setup_method = "init_some_levels"

ActiveThorns = "TOVSolver"

ADMBase::initial_data            = "tov"
ADMBase::initial_lapse           = "tov"
ADMBase::initial_shift           = "tov"
ADMBase::initial_dtlapse         = "zero"
ADMBase::initial_dtshift         = "zero"

TOVSolver::TOV_Rho_Central[0] = 1.28e-3
TOVSolver::TOV_Gamma          = 2.0
TOVSolver::TOV_K              = 100.0


IOBasic::outInfo_every              = 1
IOBasic::outInfo_vars               = "HydroBase::rho ADMBase::lapse"

IO::out_dir      = $parfile

IOScalar::outScalar_every = 32
IOScalar::one_file_per_group = yes
IOScalar::outScalar_vars  = "
 HydroBase::rho
 HydroBase::press
 HydroBase::eps
 HydroBase::vel
 ADMBase::lapse
 ADMBase::metric
 ADMBase::curv
 ML_ADMConstraints::ML_Ham
 ML_ADMConstraints::ML_mom
 
"

IOASCII::out1D_every     = 256
IOASCII::one_file_per_group = yes
IOASCII::output_symmetry_points = no
IOASCII::out3D_ghosts           = no
IOASCII::out3D_outer_ghosts     = no
IOASCII::out1D_vars      = "
 HydroBase::rho
 HydroBase::press
 HydroBase::eps
 HydroBase::vel
 ADMBase::lapse
 ADMBase::metric
 ADMBase::curv
 ML_ADMConstraints::ML_Ham
 ML_ADMConstraints::ML_mom
 
"


Overwriting par/static_tov_small.par


Next create a shortcut variable for the directory where simulation output data is dumped. To do this, edit and execute the following line.

In [59]:
sim_dir = '/home/ejwest/ETK/simulations'

To delete any old versions of the simulation and start over again, uncomment and execute the following line.

In [64]:
!rm -rf $sim_dir/static_tov_small

At long last, we are ready run Cactus. This configuration specifies running on two threads, with 1 thread per process. To execute this command, Cactus uses a "RunScript" stored in configs/sim/RunScript. You might want to take a look at it. Identifiers sandwiched between @ symbols get replaced by Simfactory prior to execution. To view the RunScript using %pycat, uncomment and execute the following lines.

In [65]:
#%pycat $cactus_dir/configs/sim/RunScript

Enough already! Let's run Cactus! We are creating a new simulation called "static_tov_small", using the above parfile, and the specified number of processes. Set procs equal to the number of logical cores on your machine (number of physical cores times 2 if your CPU allows hyperthreading, otherwise just the number of physical cores). To run, uncomment and execute the following lines. The simulation make take several hours to finish.

In [66]:
%cd $cactus_dir
!./simfactory/bin/sim create-run static_tov_small --parfile=par/static_tov_small.par --procs=2

/home/ejwest/ETK/Cactus
Parameter file: /home/ejwest/ETK/Cactus/par/static_tov_small.par
Skeleton Created
Job directory: "/home/ejwest/ETK/simulations/static_tov_small"
Executable: "/home/ejwest/ETK/Cactus/exe/cactus_sim"
Option list: "/home/ejwest/ETK/simulations/static_tov_small/SIMFACTORY/cfg/OptionList"
Submit script: "/home/ejwest/ETK/simulations/static_tov_small/SIMFACTORY/run/SubmitScript"
Run script: "/home/ejwest/ETK/simulations/static_tov_small/SIMFACTORY/run/RunScript"
Parameter file: "/home/ejwest/ETK/simulations/static_tov_small/SIMFACTORY/par/static_tov_small.par"
Simulation name: static_tov_small
Assigned restart id: 0 
Running simulation static_tov_small
Preparing:
+ set -e
+ cd /home/ejwest/ETK/simulations/static_tov_small/output-0000-active
+ echo Checking:
Checking:
+ pwd
/home/ejwest/ETK/simulations/static_tov_small/output-0000-active
+ hostname
newton
+ date
Sat Apr 21 01:31:39 CDT 2018
+ echo Environment:
Environment:
+ export CACTUS_NUM_PROCS=1
+ export CACTUS_NU

INFO (Carpet): Multi-Model listing:
   model 0: "world"
INFO (Carpet): Multi-Model process distribution:
   process 0: model 0 "world"
INFO (Carpet): Multi-Model: This is process 0, model 0 "world"
INFO (CarpetLib): Process startup time was 1.38 seconds
Current core file size limit: hard=[unlimited], soft=0 MB
Current memory size limit: hard=[unlimited], soft=[unlimited]
INFO (CycleClock): Measuring CycleClock tick via OpenMP...
INFO (CycleClock): Calibrated CycleClock: 0.303697 ns per clock tick (3.29275 GHz)
INFO (Formaline): Configuration id: config-sim-newton-home-ejwest-ETK-Cactus
INFO (Formaline): Build id: build-sim-newton-ejwest-2018.04.09-14.43.34-32328
INFO (Formaline): Simulation id: run-static_tov_small-newton-ejwest-2018.04.21-06.31.40-17522
INFO (Formaline): Run id: run-static_tov_small-newton-ejwest-2018.04.21-06.31.40-17522
INFO (Vectors): Using vector size 2 for architecture SSE2 (64-bit precision)
-----------------------------------------------------------------------

INFO (Carpet): Grid structure (superregions, grid points):
   [0][0][0]   exterior: [0,0,0] : [25,25,25]   ([26,26,26] + PADDING) 17576
   [1][0][0]   exterior: [3,3,3] : [37,37,37]   ([35,35,35] + PADDING) 42875
   [2][0][0]   exterior: [9,9,9] : [43,43,43]   ([35,35,35] + PADDING) 42875
   [3][0][0]   exterior: [21,21,21] : [54,54,54]   ([34,34,34] + PADDING) 39304
   [4][0][0]   exterior: [45,45,45] : [77,77,77]   ([33,33,33] + PADDING) 35937
INFO (Carpet): Grid structure (superregions, coordinates):
   [0][0][0]   exterior: [-36.000000000000000,-36.000000000000000,-36.000000000000000] : [264.000000000000000,264.000000000000000,264.000000000000000] : [12.000000000000000,12.000000000000000,12.000000000000000]
   [1][0][0]   exterior: [-18.000000000000000,-18.000000000000000,-18.000000000000000] : [186.000000000000000,186.000000000000000,186.000000000000000] : [6.000000000000000,6.000000000000000,6.000000000000000]
   [2][0][0]   exterior: [-9.000000000000000,-9.000000000000000,-9.000

     1312     7.688 |    0.6701278    0.9966375 | 9.996707e-11    0.0012622
     1344     7.875 |    0.6701379    0.9966375 | 9.999455e-11    0.0012622
     1376     8.062 |    0.6701488    0.9966375 | 9.996799e-11    0.0012623
     1408     8.250 |    0.6701606    0.9966375 | 9.999928e-11    0.0012623
     1440     8.438 |    0.6701734    0.9966375 | 9.994016e-11    0.0012623
     1472     8.625 |    0.6701870    0.9966375 | 9.999302e-11    0.0012624
     1504     8.812 |    0.6702014    0.9966375 | 9.997715e-11    0.0012624
     1536     9.000 |    0.6702165    0.9966375 | 1.000000e-10    0.0012624
     1568     9.188 |    0.6702321    0.9966375 | 9.997716e-11    0.0012624
     1600     9.375 |    0.6702479    0.9966375 | 9.999492e-11    0.0012624
     1632     9.562 |    0.6702639    0.9966375 | 9.994014e-11    0.0012624
     1664     9.750 |    0.6702799    0.9966375 | 9.999812e-11    0.0012623
     1696     9.938 |    0.6702958    0.9966375 | 9.996668e-11    0.0012623
     1728   

     4160    24.375 |    0.6727066    0.9966376 | 9.998349e-11    0.0012429
     4192    24.562 |    0.6727483    0.9966376 | 9.990534e-11    0.0012426
     4224    24.750 |    0.6727902    0.9966376 | 9.999488e-11    0.0012423
     4256    24.938 |    0.6728322    0.9966376 | 9.981094e-11    0.0012420
     4288    25.125 |    0.6728740    0.9966376 | 9.998463e-11    0.0012418
     4320    25.312 |    0.6729154    0.9966376 | 9.988160e-11    0.0012415
     4352    25.500 |    0.6729565    0.9966376 | 9.999863e-11    0.0012412
     4384    25.688 |    0.6729973    0.9966376 | 9.984541e-11    0.0012409
     4416    25.875 |    0.6730382    0.9966376 | 9.998510e-11    0.0012406
     4448    26.062 |    0.6730795    0.9966376 | 9.994634e-11    0.0012403
---------------------------------------------------------------------------
Iteration      Time |              ADMBASE::alp |            HYDROBASE::rho
                    |      minimum      maximum |      minimum      maximum
------------

...

    71808   420.750 |    0.6728864    0.9966329 | 9.988588e-11    0.0012381
    71840   420.938 |    0.6728856    0.9966328 | 9.934423e-11    0.0012382
    71872   421.125 |    0.6728848    0.9966328 | 9.975530e-11    0.0012382
    71904   421.312 |    0.6728838    0.9966327 | 9.954571e-11    0.0012382
    71936   421.500 |    0.6728826    0.9966327 | 9.997027e-11    0.0012382
    71968   421.688 |    0.6728811    0.9966327 | 9.951310e-11    0.0012382
    72000   421.875 |    0.6728794    0.9966326 | 9.977736e-11    0.0012382
    72032   422.062 |    0.6728775    0.9966326 | 9.952836e-11    0.0012382
    72064   422.250 |    0.6728754    0.9966325 | 9.985292e-11    0.0012382
    72096   422.438 |    0.6728732    0.9966325 | 9.925421e-11    0.0012382
    72128   422.625 |    0.6728710    0.9966325 | 9.977112e-11    0.0012382
    72160   422.812 |    0.6728689    0.9966324 | 9.950768e-11    0.0012382
    72192   423.000 |    0.6728669    0.9966324 | 1.000000e-10    0.0012382
    72224   

    74656   437.438 |    0.6730700    0.9966300 | 9.944166e-11    0.0012365
    74688   437.625 |    0.6730751    0.9966300 | 9.975776e-11    0.0012364
    74720   437.812 |    0.6730805    0.9966300 | 9.944761e-11    0.0012363
    74752   438.000 |    0.6730865    0.9966300 | 1.000000e-10    0.0012363
    74784   438.188 |    0.6730932    0.9966301 | 9.946615e-11    0.0012362
    74816   438.375 |    0.6731006    0.9966301 | 9.980773e-11    0.0012362
    74848   438.562 |    0.6731086    0.9966301 | 9.928824e-11    0.0012361
---------------------------------------------------------------------------
Iteration      Time |              ADMBASE::alp |            HYDROBASE::rho
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
    74880   438.750 |    0.6731170    0.9966302 | 9.993610e-11    0.0012361
    74912   438.938 |    0.6731255    0.9966302 | 9.946545e-11    0.0012360
    74944   

    77472   453.938 |    0.6737771    0.9966369 | 9.924576e-11    0.0012300
    77504   454.125 |    0.6737874    0.9966369 | 9.983212e-11    0.0012299
    77536   454.312 |    0.6737978    0.9966370 | 9.937728e-11    0.0012298
    77568   454.500 |    0.6738084    0.9966371 | 9.998630e-11    0.0012298
    77600   454.688 |    0.6738189    0.9966372 | 9.940094e-11    0.0012297
    77632   454.875 |    0.6738291    0.9966373 | 9.980954e-11    0.0012296
    77664   455.062 |    0.6738389    0.9966373 | 9.938643e-11    0.0012295
    77696   455.250 |    0.6738483    0.9966374 | 9.993772e-11    0.0012295
    77728   455.438 |    0.6738573    0.9966375 | 9.923717e-11    0.0012294
    77760   455.625 |    0.6738662    0.9966376 | 9.971450e-11    0.0012293
    77792   455.812 |    0.6738749    0.9966376 | 9.932610e-11    0.0012292
    77824   456.000 |    0.6738838    0.9966377 | 1.000000e-10    0.0012292
    77856   456.188 |    0.6738929    0.9966378 | 9.932653e-11    0.0012291
    77888   

    80320   470.625 |    0.6744829    0.9966425 | 9.975782e-11    0.0012244
    80352   470.812 |    0.6744882    0.9966426 | 9.895979e-11    0.0012244
    80384   471.000 |    0.6744931    0.9966426 | 1.000000e-10    0.0012243
    80416   471.188 |    0.6744976    0.9966427 | 9.926722e-11    0.0012243
    80448   471.375 |    0.6745020    0.9966428 | 9.973252e-11    0.0012242
    80480   471.562 |    0.6745066    0.9966428 | 9.935596e-11    0.0012242
    80512   471.750 |    0.6745113    0.9966429 | 9.992390e-11    0.0012241
    80544   471.938 |    0.6745165    0.9966430 | 9.896202e-11    0.0012241
    80576   472.125 |    0.6745221    0.9966431 | 9.981960e-11    0.0012241
    80608   472.312 |    0.6745283    0.9966431 | 9.941673e-11    0.0012240
---------------------------------------------------------------------------
Iteration      Time |              ADMBASE::alp |            HYDROBASE::rho
                    |      minimum      maximum |      minimum      maximum
------------

---------------------------------------------------------------------------
Iteration      Time |              ADMBASE::alp |            HYDROBASE::rho
                    |      minimum      maximum |      minimum      maximum
---------------------------------------------------------------------------
    83200   487.500 |    0.6747928    0.9966456 | 9.996258e-11    0.0012222
    83232   487.688 |    0.6747960    0.9966456 | 9.943899e-11    0.0012222
    83264   487.875 |    0.6747988    0.9966456 | 9.967137e-11    0.0012222
    83296   488.062 |    0.6748010    0.9966456 | 9.942005e-11    0.0012222
    83328   488.250 |    0.6748022    0.9966456 | 9.993202e-11    0.0012222
    83360   488.438 |    0.6748025    0.9966456 | 9.956769e-11    0.0012222
    83392   488.625 |    0.6748019    0.9966455 | 9.978002e-11    0.0012222
    83424   488.812 |    0.6748005    0.9966455 | 9.969899e-11    0.0012222
    83456   489.000 |    0.6747987    0.9966455 | 1.000000e-10    0.0012222
    83488   

Once the simulation has ended, move to the directory where the output data has been written.

In [67]:
%cd $sim_dir/static_tov_small/output-0000/static_tov_small

/home/ejwest/ETK/simulations/static_tov_small/output-0000/static_tov_small


Browse the data files.

In [68]:
%ls *.asc

admbase-curv.average.asc      hydrobase-press.minimum.asc
admbase-curv.count.asc        hydrobase-press.norm1.asc
admbase-curv.d.asc            hydrobase-press.norm2.asc
admbase-curv.iaverage.asc     hydrobase-press.norm_inf.asc
admbase-curv.icount.asc       hydrobase-press.sum.asc
admbase-curv.inorm1.asc       hydrobase-press.x.asc
admbase-curv.inorm2.asc       hydrobase-press.y.asc
admbase-curv.maximum.asc      hydrobase-press.z.asc
admbase-curv.minimum.asc      hydrobase-rho.average.asc
admbase-curv.norm1.asc        hydrobase-rho.count.asc
admbase-curv.norm2.asc        hydrobase-rho.d.asc
admbase-curv.norm_inf.asc     hydrobase-rho.iaverage.asc
admbase-curv.sum.asc          hydrobase-rho.icount.asc
admbase-curv.x.asc            hydrobase-rho.inorm1.asc
admbase-curv.y.asc            hydrobase-rho.inorm2.asc
admbase-curv.z.asc            hydrobase-rho.maximum.asc
admbase-lapse.average.asc     hydrobase-rho.minimum.asc
admbase-lapse.count.asc       hydrobase-rho.norm1.asc
admbase-lapse

Let's take a look at one of these files, say hydrobase-rho.maximum.asc, using %pycat.

In [69]:
%pycat hydrobase-rho.maximum.asc

The top of the file indicates what the columns mean:

HYDROBASE::rho (hydrobase-rho)
* 1:iteration
* 2:time, how much time has passed in the simulation
* 3:data, in this case the variable rho

Once we know all this, it is straightforward to plot the data.

Enable plotting in the notebook using matplotlib and numpy.

In [70]:
%matplotlib inline
import numpy as np #useful for importing data into arrays
import matplotlib.pyplot as plt

Import data into an ndarray (numpy array), and define variables for the different columns.

In [71]:
data = np.genfromtxt("hydrobase-rho.maximum.asc")
itnum = data[:,0]
time = data[:,1]
rhomax = data[:,2]

Then plot rhomax versus time.

In [72]:
fig, ax = plt.subplots(1,1)
ax.plot(time, rhomax)
ax.set_title('Maximum mass density')
ax.set_xlabel('time')
ax.set_ylabel('rho')
plt.show()

Congratulations! You have setup Cactus, run your first simulation, and visualized some of the results! 